In [67]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

In [68]:
import numpy as np

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.5 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [69]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'])

In [70]:
# Split the data: December 2022 as test data
train_data = data[data['date'] < '2023-02-01']
test_data = data[data['date'] >= '2023-02-01']

In [71]:

BCV = BlockingTimeSeriesSplit(n_splits=5)

In [72]:
# Collecting all data list
test_list = []

# Lists to store the MSE for each blocked split
blocked_mse_list = []
blocked_rmse_list = []

In [76]:
# forschleife, um die 5 Blöcke zu durchlaufen
for train_block, val_block in BCV.split(train_data):
    # Split the data into training and validation sets
    train_fold, val_fold= train_data.loc[train_block], train_data.iloc[val_block]

    model = ARIMA(train_fold['count'], order=(1,1,1))

    # Fit the model
    model_fit = model.fit()

    # Make predictions on the validation fold
    val_fold['prediction'] = model_fit.forecast(len(val_fold))

    val_fold = val_fold.dropna()

    # Calculate the MSE for the validation fold
    mse = mean_squared_error(val_fold['count'], val_fold['prediction'])
    rmse = np.sqrt(mse)

    # Append the MSE to the list
    blocked_mse_list.append(mse)
    blocked_rmse_list.append(rmse)

    

/var/folders/q3/tk8ksh9n6233c6nj3njq87x80000gn/T/ipykernel_8531/2097049221.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_fold['prediction'] = model_fit.forecast(len(val_fold))
/Users/paddy/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/paddy/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/paddy/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index 

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.